In [ ]:
!nvcc --version

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%cu
#include <cstdio>
#include <string>
#include <iostream>
#include <random>
#include <iomanip>
#include <cmath>
#include <bits/stdc++.h>

using namespace std;

__device__ int partition(int *arr, int start, int end)
{
  int index = 0, pivotElement = arr[end], pivotIndex;
  int *temp = new int[end - start + 1];
  for (int i = start; i <= end; i++)
  {
    if (arr[i] < pivotElement)
    {
      temp[index] = arr[i];
      index++;
    }
  }

  temp[index] = pivotElement; // pushing pivotElement in temp
  index++;

  for (int i = start; i < end; i++) // pushing all the elements in temp which are greater than pivotElement
  {
    if (arr[i] > pivotElement)
    {
      temp[index] = arr[i];
      index++;
    }
  }

  index = 0;
  for (int i = start; i <= end; i++) // copying all the elements to original array i.e arr
  {
    if (arr[i] == pivotElement)
    {
      pivotIndex = i;
    }
    arr[i] = temp[index];
    index++;
  }
  return pivotIndex; // returning pivotIndex
}

__device__ void quickSort(int *arr, int start, int end)
{
  if (start < end)
  {
    int partitionIndex = partition(arr, start, end);
    quickSort(arr, start, partitionIndex - 1);
    quickSort(arr, partitionIndex + 1, end);
  }
  return;
}

__global__ void parallel_quicksort(int *values, int n_threads, int range)
{
  int start = n_threads * threadIdx.x;

  int *interval = new int[range];

  for (int i = 0; i < range; i++)
  {
    int pos = start + i;
    interval[i] = values[pos];
  }

  quickSort(interval, 0, range - 1);

  for (int i = 0; i < range; i++)
  {
    int pos = start + i;
    values[pos] = interval[i];
  }
}

int main()
{
  int *host_values;
  int *device_values;

  int n = 121;
  int n_threads = sqrt(n);

  int size = n * sizeof(int);

  host_values = (int *)malloc(size);
  cudaMalloc((void **)&device_values, size);

  for (int i = 0; i < n; i++)
  {
    host_values[i] = rand() % 1000;
  }

  int rest = n - ((n_threads - 1) * n_threads);
  int range;

  if (n % n_threads == 0)
  {
    range = n_threads;
  }
  else
  {
    range = n_threads + round(rest / n_threads);
  }
  cout << "n_threads -> " << n_threads;
  cout << "\nrange -> " << range;

  cout << "\nInicio \n\n";

  for (int i = 0; i < n; i++)
  {
    cout << host_values[i] << ", ";
  }

  cudaMemcpy(device_values, host_values, size, cudaMemcpyHostToDevice);

  parallel_quicksort<<<1, n_threads>>>(device_values, n_threads, range);

  cudaMemcpy(host_values, device_values, size, cudaMemcpyDeviceToHost);

  cout << "\n\nFim \n\n";

  int count = 0;
  for (int i = 0; i < n; i++)
  {
    int start = range * i;

    if (i != 0 && count % range == 0)
    {
      cout << "\n";
      count = 0;
    }

    cout << host_values[i] << ", ";
    count++;
  }

  cudaFree(device_values);

  return 0;
}